In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}
cols = 'trade name qty price active reason market'.split()
cols

['trade', 'name', 'qty', 'price', 'active', 'reason', 'market']

### After there is change in orders

In [2]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.shape[0]

27

In [3]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

orders[cols].to_csv(output_file, header=False, index=False)
orders[cols].to_csv(data_file, header=False, index=False)
orders[cols].to_csv(box_file, header=False, index=False)

### After call ord-log

In [4]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [5]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

In [6]:
from itables import show
show(df[['trans','name','spd','reason','qty','target','current','chg','percent']])

<IPython.core.display.Javascript object>

trans,name,spd,reason,qty,target,current,chg,percent


In [7]:
name = 'SYNEX'
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE name = '%s'
'''
sql = sql % name
order = pd.read_sql(sql, conlite)
order.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,SYNEX,"6,000",31.25,"187,500.00",DFM10%,SET100,0


In [8]:
price = 31.25
qty = 6000
sql = '''
UPDATE orders
SET price = %s, qty = %s
WHERE name = '%s'
'''
sql = sql % (price, qty, name)
rp = conlite.execute(sql)
rp.rowcount

1

### End of Update Process

In [9]:
sql = '''
UPDATE stocks SET buy_target=(
SELECT orders.price 
FROM orders 
WHERE orders.name=stocks.name AND orders.trade='B')'''
rp = conlite.execute(sql)
rp.rowcount

50

In [10]:
sql = '''
UPDATE stocks SET sell_target=(
SELECT orders.price 
FROM orders 
WHERE orders.name=stocks.name AND orders.trade='S')'''
rp = conlite.execute(sql)
rp.rowcount

50

In [11]:
sql = '''
UPDATE stocks SET qty = (
SELECT orders.qty 
FROM orders 
WHERE orders.name = stocks.name AND orders.trade = 'S')'''
rp = conlite.execute(sql)
rp.rowcount

50

In [12]:
sql = '''
UPDATE stocks
SET buy_target = 0
WHERE buy_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

40

In [13]:
sql = '''
UPDATE stocks
SET sell_target = 0
WHERE sell_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

33

In [14]:
sql = '''
UPDATE stocks
SET available_qty = 0
WHERE available_qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [15]:
sql = '''
UPDATE stocks
SET cost = 0
WHERE cost IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [16]:
sql = '''
UPDATE stocks
SET qty = 0
WHERE qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

33